In [76]:

import numpy as np


In [77]:
dfcustomers = pd.read_csv('Data\olist_customers_dataset.csv')
dfgeolocation = pd.read_csv('Data\olist_geolocation_dataset.csv')
dforder_items = pd.read_csv('Data\olist_order_items_dataset.csv')
dforders = pd.read_csv('Data\olist_orders_dataset.csv')
dforder_payments = pd.read_csv('Data\olist_order_payments_dataset.csv')
dforder_reviews = pd.read_csv('Data\olist_order_reviews_dataset.csv')
dfproducts = pd.read_csv('Data\olist_products_dataset.csv')
dfsellers = pd.read_csv('Data\olist_sellers_dataset.csv')
dfproduct_category_name = pd.read_csv('Data\product_category_name_translation.csv')

In [78]:
dforder_items_prduct_cat =  dforder_items.merge(dfproducts, how='inner', left_on=["product_id"], right_on=["product_id"])
dforder_items_prduct_cat = dforder_items_prduct_cat[['order_id','product_category_name']]

In [82]:
dforders = dforders[['order_id','customer_id']]

In [83]:
dforders_items = dforders.merge(dforder_items_prduct_cat, how='inner', left_on=["order_id"], right_on=["order_id"])
# dforders_items = dforders_items[['customer_id','product_id']]

In [84]:
# dforders_items_grouped = dforders_items.groupby(by=['customer_id','product_category_name']).count().reset_index()

In [85]:
dforders_items = dforders_items[['customer_id','product_category_name']]

In [86]:
data = pd.melt(dforders_items.set_index('customer_id')['product_category_name'].apply(pd.Series).reset_index(), 
             id_vars=['customer_id'],
             value_name='product_category_name') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customer_id', 'product_category_name']) \
    .agg({'product_category_name': 'count'}) \
    .rename(columns={'product_category_name': 'purchase_count'}) \
    .reset_index()

In [88]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customer_id', columns='product_category_name')

In [89]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [90]:
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['customer_id'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(97933, 3)


,customer_id,product_category_name,scaled_purchase_freq
170,0074a15febc9459ab2f999ea0e22c8d6,agro_industria_e_comercio,0.0
398,01122215dd21ac872ae567ec4e351e01,agro_industria_e_comercio,0.0
677,01c79a7d3c3d25b655514206df838701,agro_industria_e_comercio,0.0
1862,04d5cb41e92cebd0f1051d38c10f5ec4,agro_industria_e_comercio,0.0
2023,05455dfa7cd02f13d132aa7a6a9729c6,agro_industria_e_comercio,1.0


In [91]:
# def normalize_data(data):
#     df_matrix = pd.pivot_table(data, values='purchase_count', index='customer_id', columns='product_category_name')
#     df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
#     d = df_matrix_norm.reset_index()
#     d.index.names = ['scaled_purchase_freq']
#     return pd.melt(d, id_vars=['customer_id'], value_name='scaled_purchase_freq').dropna()